In [5]:
import cv2
import numpy as np
import argparse
import glob
from utils.opv import OpvModel
mymodel2 = OpvModel('face-detection-adas-0001',device = 'CPU',fp='FP32')

C:\ProgramData\Anaconda3\envs\Brain_AI\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\Intel_JS\openVINO\utils\opv.py:61: DeprecationWarning: Reading network using constructor is deprecated. Please, use IECore.read_network() method instead
  net = IENetwork(model=model_xml, weights=model_bin)
C:\Intel_JS\openVINO\utils\opv.py:65: DeprecationWarning: IEPlugin class is deprecated. Please use IECore class instead.
  plugin = IEPlugin(device=device)


In [6]:
friends = cv2.imread('images/friends.JPEG')
friends = cv2.resize(friends, (600, 400))
predictions = mymodel2.Predict(friends)

C:\ProgramData\Anaconda3\envs\Brain_AI\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
GENDERS_FOR_OPENVINO = ['Female', 'Male']

C:\ProgramData\Anaconda3\envs\Brain_AI\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
args= {
    'xml': 'models/age-gender-recognition-retail-0013/FP32/age-gender-recognition-retail-0013.xml',
    'bin': 'models/age-gender-recognition-retail-0013/FP32/age-gender-recognition-retail-0013.bin'
}
openvino_net = cv2.dnn.readNet(args["xml"], args["bin"])
openvino_net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

C:\ProgramData\Anaconda3\envs\Brain_AI\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
def DrawBoundingBoxes(predictions, image, conf=0.8):
    mymodel1=OpvModel("age-gender-recognition-retail-0013",device="CPU",fp="FP32")
    
    canvas=image.copy()
    predictions_1=predictions[0][0] # numpy array 타입
    confidence=predictions_1[:,2]   # 2번 데이터만
    topresults=predictions_1[(confidence>conf)]  # conf 보다 큰 데이터만
    (h,w)=canvas.shape[:2]
    for detection in topresults:
        box=detection[3:7]*np.array([w,h,w,h])
        (xmin,ymin,xmax,ymax)=box.astype("int")
        img=canvas[ymin:ymax,xmin:xmax].copy()
        blob = cv2.dnn.blobFromImage(img,size=(62,62), ddepth=cv2.CV_8U)
        openvino_net.setInput(blob)
        detections=openvino_net.forwardAndRetrieve(['prob', 'age_conv3'])
        gender = GENDERS_FOR_OPENVINO[detections[0][0][0].argmax()]
        age = detections[1][0][0][0][0][0] * 100
        cv2.rectangle(canvas,(xmin,ymin),(xmax,ymax),(0,0,255),4)
        text = f"{gender}, {age:.0f}years old"
        if gender == 'Male':
            color = (255, 0, 0)
        else:
            color = (0, 0, 255)
        cv2.putText(canvas, text, (xmin, ymin), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
        #cv2.putText(canvas,emo+" "+str(round(max_emo_val*100,1))+"%",(xmin,ymin),cv2.FONT_HERSHEY_SIMPLEX,0.6,(255,0,0),2)
    #cv2.putText(canvas,str(len(topresults))+" face(s) detected",(50,50),cv2.FONT_HERSHEY_SIMPLEX,0.6,(255,0,0),2)
    
    return canvas

C:\ProgramData\Anaconda3\envs\Brain_AI\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
cv2.imshow("canvas",DrawBoundingBoxes(predictions,friends))
cv2.waitKey(0)
cv2.destroyAllWindows()

C:\ProgramData\Anaconda3\envs\Brain_AI\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\Intel_JS\openVINO\utils\opv.py:61: DeprecationWarning: Reading network using constructor is deprecated. Please, use IECore.read_network() method instead
  net = IENetwork(model=model_xml, weights=model_bin)
C:\Intel_JS\openVINO\utils\opv.py:65: DeprecationWarning: IEPlugin class is deprecated. Please use IECore class instead.
  plugin = IEPlugin(device=device)


In [15]:
camera = cv2.VideoCapture(0)

while True:
    ret, frame = camera.read()
    
    if not ret:
        break
    predictions = mymodel2.Predict(frame)
    cv2.imshow('cam', DrawBoundingBoxes(predictions, frame))
    
    if cv2.waitKey(0)&0xFF == 27:
        break
camera.release()
cv2.destroyAllWindows()

C:\ProgramData\Anaconda3\envs\Brain_AI\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


ValueError: could not broadcast input array from shape (1,3,384,672) into shape (1,3,62,62)